In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
tf.enable_eager_execution()

In [0]:
class DenseUnit(tf.keras.Model):
    def __init__(self, filter_in, filter_out):
        super(DenseUnit, self).__init__()
        self.bn1 = layers.BatchNormalization()
        self.conv1 = layers.Conv2D(filter_in, 1, 1, padding='same')
        self.bn2 = layers.BatchNormalization()
        self.conv2 = layers.Conv2D(filter_out, 3, 1, padding='same')
        self.concat = layers.Concatenate()
    def __call__(self, x, training=False, mask=None):   # x: (Batch, H, W, filter_in)
        y = self.bn1(x, training=training)
        y = layers.Activation('relu')(y)
        y = self.conv1(y)    
        y = self.bn2(y, training=training)
        y = layers.Activation('relu')(y)
        y = self.conv2(y)                              # y: (Batch, H, W, filter_out)
        return self.concat([x,y])                      # (Batch, H, W, filter_in+filter_out)

In [0]:
class Transition(tf.keras.Model):          #채널 수가 급격히 증가한 경우 줄여주는 역할
    def __init__(self, filter_out, kernel_size):
        super(Transition, self).__init__()
        self.conv = layers.Conv2D(filter_out, kernel_size, padding='same')
        self.pool = layers.MaxPool2D()
    def __call__(self, x, training=False, mask=None):
        x = self.conv(x)
        x = self.pool(x)
        return x

In [0]:
class DenseNet(tf.keras.Model):
    def __init__(self, num_classes):
        super(DenseNet, self).__init__()
        self.conv1 = layers.Conv2D(8, 3, padding='same', activation='relu') #(H, W, 8)
        self.dl1 = DenseUnit(8, 8)      #(H, W, 16)
        self.tl1 = Transition(16, 3)    #(H/2, W/2, 16)
        self.dl2 = DenseUnit(16, 16)      #(H/2, W/2, 32)
        self.tl2 = Transition(32, 3)      #(H/4, W/4, 32) 
        self.dl3 = DenseUnit(32, 32)      #(H/4, W/4, 64)
        
        self.flatten = layers.Flatten()
        self.dense1 = layers.Dense(128, activation='relu')
        self.dense2 = layers.Dense(num_classes, activation='softmax')
    def __call__(self, x, training=False, mask=None):
        x = self.conv1(x)
        x = self.dl1(x, training=training)
        x = self.tl1(x)
        x = self.dl2(x, training=training)
        x = self.tl2(x)
        x = self.dl3(x, training=training)
        x = self.flatten(x)
        x = self.dense1(x)
        return self.dense2(x)

In [0]:
@tf.function
def train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy):
  with tf.GradientTape() as tape:
    predictions = model(images, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))
  train_loss(loss)
  train_accuracy(labels, predictions)

@tf.function
def test_step(model, images, labels, loss_object, test_loss, test_accuracy):
  predictions = model(images,training=False)
  t_loss = loss_object(labels, predictions)
  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [0]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
train_x = (x_train / 255.).astype(np.float32)
test_x = (x_test / 255.).astype(np.float32)
train_y = tf.keras.utils.to_categorical(y_train)
test_y = tf.keras.utils.to_categorical(y_test)

In [0]:
train_ds = tf.data.Dataset.from_tensor_slices((train_x, train_y)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((test_x, test_y)).batch(32)

In [0]:
def lr_schedule(epoch):
  lr = 1e-3
  if epoch > 180:
      lr *= 0.5e-3
  elif epoch > 160:
      lr *= 1e-3
  elif epoch > 120:
      lr *= 1e-2
  elif epoch > 80:
      lr *= 1e-1
  print('Learning rate: ', lr)
  return lr

In [9]:
model = DenseNet(10)

loss_object = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(lr_schedule(0))

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')
test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.CategoricalAccuracy(name='test_accuracy')

Learning rate:  0.001


In [0]:
for epoch in range(100):
  for images, labels in train_ds:
    train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy)
  for t_images, t_labels in test_ds:
    test_step(model, t_images, t_labels, loss_object, test_loss, test_accuracy)
  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print(template.format(epoch + 1,
                        train_loss.result(),
                        train_accuracy.result() * 100,
                        test_loss.result(),
                        test_accuracy.result() * 100))
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

Epoch 1, Loss: 1.2214072942733765, Accuracy: 56.571258544921875, Test Loss: 0.9933087825775146, Test Accuracy: 65.19000244140625
Epoch 2, Loss: 0.8617086410522461, Accuracy: 69.5999984741211, Test Loss: 0.9665783047676086, Test Accuracy: 66.97999572753906
Epoch 3, Loss: 0.7057992815971375, Accuracy: 75.24799346923828, Test Loss: 1.1319159269332886, Test Accuracy: 66.43000030517578
Epoch 4, Loss: 0.5845091938972473, Accuracy: 79.24800109863281, Test Loss: 1.1685810089111328, Test Accuracy: 66.68000030517578
Epoch 5, Loss: 0.4806031286716461, Accuracy: 82.78399658203125, Test Loss: 1.3094109296798706, Test Accuracy: 67.04999542236328
Epoch 6, Loss: 0.399684876203537, Accuracy: 85.49400329589844, Test Loss: 1.4509330987930298, Test Accuracy: 66.56999969482422
Epoch 7, Loss: 0.34745892882347107, Accuracy: 87.58200073242188, Test Loss: 1.4750522375106812, Test Accuracy: 66.79999542236328
Epoch 8, Loss: 0.29399776458740234, Accuracy: 89.44000244140625, Test Loss: 1.4271347522735596, Test Acc